### Preamble

In [3]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


# Topic 6. Higher-order functions and modular programming

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [25]:
// sum of elements
def divideAndConquer[A, B](l: List[A])
        (nil: B)
        (cons: (A, B) => B): B = 
    l match 
        case Nil => nil : B
        case head :: (tail: List[A]) => 
            val tailSol: B = divideAndConquer(tail)(nil)(cons)
            cons(head, tailSol) : B

defined function divideAndConquer

In [28]:
// sum of elements
def sum(l: List[Int]): Int = 
    divideAndConquer(l)
        (0 : Int)
        ((head, tailSol) => 
            head + tailSol : Int)

defined function sum

In [30]:
// sum of elements
def sum(l: List[Int]): Int = 
    l.foldRight
        (0 : Int)
        ((head, tailSol) => 
            head + tailSol : Int)

defined function sum

In [34]:
// sum of elements
def sum(l: List[Int]): Int = 
    l.foldRight(0)(_+_)

defined function sum

In [27]:
// sum of elements
def sum(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: (tail: List[Int]) => 
            val tailSol: Int = sum(tail)
            head + tailSol : Int

defined function sum

In [18]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "insertLast" should "work" in:
        insertLast(1::List(2,3), 4) shouldBe {
            val head: Int = 1
            val tailSol: List[Int] = List(2,3,4)
            head :: tailSol // List(1,2,3,4)
        }
        insertLast(List(1), 10) shouldBe List(1,10)
        insertLast(List(), 10) shouldBe List(10)
        insertLast(List(), 1) shouldBe List(1)

defined class TestInsertLast

In [29]:

def insertLast[A](l: List[A], e: A): List[A] = 
    divideAndConquer(l)
        (List(e))
        ((head: A, tailSol: List[A]) => 
            head :: tailSol : List[A])

defined function insertLast

In [31]:

def insertLast[A](l: List[A], e: A): List[A] = 
    l.foldRight
        (List(e))
        ((head: A, tailSol: List[A]) => 
            head :: tailSol : List[A])

defined function insertLast

In [32]:

def insertLast[A](l: List[A], e: A): List[A] = 
    l.foldRight
        (List(e))
        ((head, tailSol) => 
            head :: tailSol)

defined function insertLast

In [33]:

def insertLast[A](l: List[A], e: A): List[A] = 
    l.foldRight(List(e))(_ :: _)

defined function insertLast

In [14]:

def insertLast[A](l: List[A], e: A): List[A] = 
    l match 
        case Nil => ??? : List[A]
        case head :: (tail: List[A]) => 
            val tailSol: List[A] = insertLast(tail, e)
            ??? : List[A]

defined function insertLast

In [21]:

def insertLast[A](l: List[A], e: A): List[A] = 
    l match 
        case Nil => List(e)
        case head :: tail => head :: insertLast(tail, e)

defined function insertLast

In [19]:

def insertLast[A](l: List[A], e: A): List[A] = 
    l match 
        case Nil => List(e) : List[A]
        case head :: (tail: List[A]) => 
            val tailSol: List[A] = insertLast(tail, e)
            head :: tailSol : List[A]

defined function insertLast

In [20]:
run(TestInsertLast(insertLast))

cmd18$Helper$TestInsertLast:
insertLast
- should work


In [22]:

def insertLast[A](l: List[A]): A => List[A] = 
    l match 
        case Nil => (e: A) => List(e)
        case head :: (tail: List[A]) => e => 
            val tailSol: List[A] = insertLast(tail)(e)
            head :: tailSol : List[A]

defined function insertLast

In [23]:
run(TestInsertLast((l, e) => insertLast(l)(e)))

cmd18$Helper$TestInsertLast:
insertLast
- should work


In [24]:
run(TestInsertLast(insertLast(_)(_)))

cmd18$Helper$TestInsertLast:
insertLast
- should work


In [3]:
// product of elements 

defined function multiply

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [4]:
// monomorphic `combine` method

defined function combine

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [ ]:
// more generically: foldRight


Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [6]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(
        0, // direct solution to atomic problem
        (head, subsol) => head + subsol // composing sub-solution
    )

defined function sum

In [7]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1, // direct solution
        (head: Int, subsol: Int) => head * subsol // compose
    )

defined function multiply

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [9]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

defined object Standard

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

In [10]:
// foldRight over lists



res10: Int = 6

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [11]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

defined function foldLeft
defined function filter
defined function map
defined function flatMap

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [12]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

In [13]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (unfortunately, the concept of catamorphism won't be explained due to lack of time). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

In [35]:
// length function, imperatively
def iterative[A, B](l: List[A]): B = 
    var out: B = ??? 
    var aux: List[A] = l 
    while aux != Nil do 
        out = ???/*(out, aux.head)*/
        aux = aux.tail
    out

defined function iterative

In [36]:
// length function, imperatively
def iterative[A, B](l: List[A]): B = 
    def step(aux: List[A], out: B): B = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(tail, ???/*(out, head)*/)

    step(l, ???)

defined function iterative

In [41]:
val l: List[Int] = List(1,2,3,4)

l: List[Int] = List(1, 2, 3, 4)

In [43]:
l.apply(2)

res43: Int = 3

In [44]:
// length function, imperatively
def iterative[A, B](l: List[A])
        (initial: B)
        (update: (B, A) => B): B = 
    def step(aux: List[A], out: B): B = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(tail, update(out, head))

    step(l, initial)

defined function iterative

In [37]:
// length function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(aux: List[A], out: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(tail, ???/*(out, head)*/)

    step(l, ???)

defined function reverse

In [38]:
// length function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(aux: List[A], out: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(tail, head :: out)

    step(l, Nil)

defined function reverse

In [45]:
// length function, imperatively
def reverse[A](l: List[A]): List[A] = 
    iterative(l)
        (Nil)
        ((out, e) => e :: out)

defined function reverse

In [52]:
// length function, imperatively
def reverse[A](l: List[A]): List[A] = 
    l.foldLeft
        (Nil)
        ((out, e) => e :: out)

defined function reverse

In [39]:
reverse(List(1,2,3,4))

res39: List[Int] = List(4, 3, 2, 1)

In [46]:
def occurrencesTR[A](l: List[A], a: A): Int = 

    def step(aux: List[A], out: Int): Int = 
        aux match 
            case Nil => out 
            case e :: tail => 
                step(tail, if e == a then 1+out else out)

    step(l, 0)

defined function occurrencesTR

In [47]:
def occurrencesIterative[A](l: List[A], a: A): Int = 
    iterative(l)
        (0)
        ((out: Int, e: A) => if e == a then 1+out else out)

defined function occurrencesIterative

In [51]:
def occurrencesIterative[A](l: List[A], a: A): Int = 
    l.foldLeft
        (0)
        ((out: Int, e: A) => if e == a then 1+out else out)

defined function occurrencesIterative

In [50]:
occurrencesIterative(List(1,1,2,3,1,34,1), 1)

res50: Int = 4

In [49]:
List('a', 'b', 'c').zipWithIndex

res49: List[(Char, Int)] = List(('a', 0), ('b', 1), ('c', 2))

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [16]:
// abstraction: foldLeft, imperatively


defined function foldLeft

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ??? 

-- [E097] Syntax Error: cell1.sc:2:4 -------------------------------------------
2 |def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
  |    ^
  |TailRec optimisation not applicable, method foldLeft contains no recursive calls
Compilation Failed

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [18]:
def length[A](list: List[A]): Int = 
    ??? 

defined function length

In [19]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [20]:
def reverse[A](list: List[A]): List[A] = 
    ??? 

defined function reverse

But we may also use the `foldRight` function to implement them:

In [21]:
def length[A](list: List[A]): Int = 
    ??? 

defined function length

In [22]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [24]:
def reverse[A](list: List[A]): List[A] = 
    ???

defined function reverse

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [26]:
// blowing up the stack 


java.lang.StackOverflowError: null

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [1]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

defined function foldRightTR

This implementation works right with big lists:

In [28]:
// check it out


res28: Int = 0

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [29]:
// foldLeftAux


defined function foldLeftAux

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [30]:
// foldLeftAux with foldRight


defined function foldLeftAux

The original signature can then be implemented as follows: 

In [2]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

defined function foldLeft

## The `map` HOF

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../../images/map.svg)

The implementation for lists can be done as follows:

In [4]:
98.toChar

res4: Char = 'b'

In [ ]:
out                           aux 
---                           ---
???                           List(97, 98, 99)
???                           List(98, 99)
???                           List(99)
List('b', 'c', 'd')           List()

In [ ]:
out                           aux 
---                           ---
Nil                           List(97, 98, 99)
'b'::Nil                      List(98, 99)
'c'::'b'::Nil                 List(99)
'd'::'c'::'b'::Nil            List()
List('b', 'c', 'd')

In [ ]:
def toChar(l: List[Int]): List[Char] = 
    def step(out: List[Char], aux: List[Int]): List[Char] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???/*(out, head)*/, tail)

    step(???, l)/*...*/

In [ ]:
def toChar(l: List[Int]): List[Char] = 
    def step(out: List[Char], aux: List[Int]): List[Char] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(head.toChar :: out, tail)

    step(Nil, l).reverse         // coste lineal

In [19]:
List(1,2,3) :+ 4

res19: List[Int] = List(1, 2, 3, 4)

In [20]:
def toChar(l: List[Int]): List[Char] = 
    def step(out: List[Char], aux: List[Int]): List[Char] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(out :+ head.toChar , tail) // coste cuadrático

    step(Nil, l)

defined function toChar

In [11]:

// map example 1
def toChar(l: List[Int]): List[Char] = 
    l match 
        case Nil => ??? : List[Char]
        case (head: Int) :: (tail: List[Int]) => 
            val tailSol: List[Char] = toChar(tail)
            ??? : List[Char]

defined function toChar

In [13]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    l match 
        case Nil => Nil : List[Char]
        case (head: Int) :: (tail: List[Int]) => 
            val tailSol: List[Char] = toChar(tail)
            head.toChar :: tailSol : List[Char]

defined function toChar

In [30]:
// map example 1
def length(l: List[String]): List[Int] = 
    l match 
        case Nil => Nil : List[Int]
        case (head: String) :: (tail: List[String]) => 
            val tailSol: List[Int] = length(tail)
            head.length :: tailSol : List[Int]

defined function length

In [32]:
// map example 1
def map[A, B](l: List[A]): List[B] = 
    l match 
        case Nil => Nil : List[B]
        case (head: A) :: (tail: List[A]) => 
            val tailSol: List[B] = map(tail)
            ???/*(head)*/ :: tailSol : List[B]

defined function map

In [33]:
// map example 1
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l match 
        case Nil => Nil : List[B]
        case (head: A) :: (tail: List[A]) => 
            val tailSol: List[B] = map(tail)(f)
            f(head) :: tailSol : List[B]

defined function map

In [34]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    map(l)((head: Int) => head.toChar)

defined function toChar

In [35]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    map(l)(_.toChar)

defined function toChar

In [36]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    l.map(_.toChar)

defined function toChar

In [37]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    var out: List[Char] = Nil
    for e <- l do 
        out = e.toChar :: out
    out.reverse

defined function toChar

In [30]:
// map example 1
def length(l: List[String]): List[Int] = 
    l match 
        case Nil => Nil : List[Int]
        case (head: String) :: (tail: List[String]) => 
            val tailSol: List[Int] = length(tail)
            head.length :: tailSol : List[Int]

defined function length

In [31]:
length(List("abc", "hola", "")) == List(3, 4, 0)

res31: Boolean = true

In [24]:
toChar(List.fill(200000)(99))

Exception in thread "SIGINT handler" java.lang.UnsupportedOperationException
	at java.base/java.lang.Thread.stop(Thread.java:1667)
	at almond.Execute.interruptible$$anonfun$1(Execute.scala:269)
	at almond.Execute.interruptible$$anonfun$adapted$1(Execute.scala:275)
	at scala.Function0.apply$mcV$sp(Function0.scala:42)
	at ammonite.repl.Signaller$$anon$1.handle(Signaller.scala:21)
	at jdk.unsupported/sun.misc.Signal$InternalMiscHandler.handle(Signal.java:198)
	at java.base/jdk.internal.misc.Signal$1.run(Signal.java:218)
	at java.base/java.lang.Thread.run(Thread.java:1583)


res24: List[Char] = List(
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
...

In [18]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    // l.divideAndConquer
    l.foldRight
        (??? : List[Char])
        ((head: Int, tailSol: List[Char]) => 
             ??? : List[Char])

defined function toChar

In [15]:
// map example 1
def toChar(l: List[Int]): List[Char] = 
    l.foldRight
        (Nil: List[Char])
        ((head: Int, tailSol: List[Char]) => 
             head.toChar :: tailSol : List[Char])

defined function toChar

In [16]:
99.toChar
100.toChar
98.toChar

res16_0: Char = 'c'
res16_1: Char = 'd'
res16_2: Char = 'b'

In [17]:
toChar(List(98,99,100)) == List('b', 'c', 'd')
toChar(List()) == List()

res17_0: Boolean = true
res17_1: Boolean = true

In [2]:
// map example 2


In [2]:
// map generically


The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [27]:
10000000000L.toChar

res27: Char = ''

In [32]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends AnyFlatSpec with should.Matchers:

    val isEven: Int => Boolean = _ % 2 == 0

    "mapping an empty list" should "return an empty list" in:
        map(List())(isEven) shouldBe ???

    "mapping an non-empty list" should "only change its content" in:
        map(List(1))(isEven) shouldBe ???
        map(List(1,2))(isEven) shouldBe ???
        map(List(1,2,3))(isEven) shouldBe ???

defined class TestMap

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [34]:
run(TestMap(map[Int, Boolean]))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


although it's not necessary to write the types explicitly (Scala will infer the types for us):

In [35]:
run(TestMap(map))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [36]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

defined function mapFR

In [37]:
run(TestMap(mapFR))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [38]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "mapping the `None` value" should "return `None`" in:
        map(None)(isEven) shouldBe ???
    
    "mapping a `Some` value" should "only change its content" in:
        map(Option(3))(isEven) shouldBe ???
        map(Option(1))(isEven) shouldBe ???
        map(Option(2))(isEven) shouldBe ???

defined class TestMapOption

The implementation is simple:

In [39]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

defined function mapOpt

In [40]:
run(TestMapOption(mapOpt))

cell38$Helper$TestMapOption:
mapping the `None` value
- should return `None`
mapping a `Some` value
- should only change its content


## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../../images/filter.svg)

In [38]:
// filter, generically 
def foo(l: List[String]): List[String] = 
    // filter those strings `s` such that `s.length > 2`
    l.filter((s: String) => s.length > 2)

defined function foo

In [40]:
// filter, generically 
def foo(l: List[String]): List[String] = 
    // filter those strings `s` such that `s.length > 2`
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[String] = foo(tail)
            if head.length > 2 then head :: tailSol
            else tailSol

defined function foo

In [47]:
def bar[A](l: List[List[A]]): List[List[A]] = 
    l match 
        case Nil => Nil
        case (head: List[A]) :: tail => 
            val tailSol: List[List[A]] = bar(tail)
            if head.length <= 2 then head :: tailSol
            else tailSol

defined function bar

In [52]:
def filter[A](l: List[A]): List[A] = 
    l match 
        case Nil => Nil
        case (head: A) :: tail => 
            val tailSol: List[A] = filter(tail)
            if ???/*(head)*/ then head :: tailSol
            else tailSol

defined function filter

In [53]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l match 
        case Nil => Nil
        case (head: A) :: tail => 
            val tailSol: List[A] = filter(tail)(pred)
            if pred(head) then head :: tailSol
            else tailSol

defined function filter

In [49]:
def bar[A](l: List[List[A]]): List[List[A]] = 
    l.filter((head: List[A]) =>
            head.length <= 2)

defined function bar

In [51]:
def bar[A](l: List[List[A]]): List[List[A]] = 
    l.filter(_.length <= 2)

defined function bar

In [50]:
bar(List(List(), List(1), List(3, 3, 3), List(2, 1)))

res50: List[List[Int]] = List(List(), List(1), List(2, 1))

In [43]:
// filter, generically 
def foo(l: List[String]): List[String] = 
    // filter those strings `s` such that `s.length > 2`
    var out: List[String] = Nil
    for e <- l do 
        if e.length > 2 then out = e :: out
    out.reverse

defined function foo

In [42]:
foo(List("aaa", "", "a"))

res42: List[String] = List("aaa")

In [39]:
foo(List("", "abc", "holaaaa"))

res39: List[String] = List("abc", "holaaaa")

In [42]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(List())(isEven) shouldBe ???
        filter(List(1))(isEven) shouldBe ???
        filter(List(1,3,5))(isEven) shouldBe ???
        filter(List(2,4,6))(isEven) shouldBe ???

defined class TestFilterList

In [43]:
run(TestFilterList(filterR))

cell42$Helper$TestFilterList:
filter
- should work


Using `foldRight` we can get a more modular implementation: 

In [45]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

defined function filterFR

In [46]:
run(TestFilterList(filterFR))

cell42$Helper$TestFilterList:
filter
- should work


### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [48]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(None)(isEven) shouldBe ???
        filter(Some(1))(isEven) shouldBe ???
        filter(Some(2))(isEven) shouldBe ???

defined class TestFilterOption

In [49]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

defined function filter

In [50]:
run(TestFilterOption(filter))

cell48$Helper$TestFilterOption:
filter
- should work


## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [55]:
class TestWords(
    words: List[String] => List[String]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in:
        words(paragraph1) shouldBe 
            List(
                "En", "un", "lugar",
                "de", "la", "Mancha", 
                "de", "cuyo", "nombre", "no",
                "quiero", "acordarme")

defined class TestWords

In [57]:
"en un lugar".split(" ").toList

res57: List[String] = List("en", "un", "lugar")

In [59]:
List[String](
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme").map[Array[String]]( 
    (sentence: String) => 
        sentence.split(" ")/*.toList : List[String]*/)

res59: List[Array[String]] = List(
  Array("En", "un", "lugar"),
  Array("de", "la", "Mancha"),
  Array("de", "cuyo", "nombre", "no"),
  Array("quiero", "acordarme")
)

In [75]:
List[String](
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    .map[List[String]]( 
        (sentence: String) => 
            sentence.split(" ").toList : List[String]
    ).flatten

res75: List[String] = List(
  "En",
  "un",
  "lugar",
  "de",
  "la",
  "Mancha",
  "de",
  "cuyo",
  "nombre",
  "no",
  "quiero",
  "acordarme"
)

In [73]:
flatten(List("En", "un", "lugar") :: 
    List(
      List("de", "la", "Mancha"),
      List("de", "cuyo", "nombre", "no"),
      List("quiero", "acordarme")
    )) == ({
    val head = List("En", "un", "lugar")
    val tailSol = List(
      "de", "la", "Mancha",
      "de", "cuyo", "nombre", "no",
      "quiero", "acordarme"
    )
    head ++ tailSol
})

scala.NotImplementedError: an implementation is missing

In [69]:
{
    val head = List("En", "un", "lugar")
    val tailSol = List(
      "de", "la", "Mancha",
      "de", "cuyo", "nombre", "no",
      "quiero", "acordarme"
    )
    head ++ tailSol
}

head: List[String] = List("En", "un", "lugar")
tailSol: List[String] = List(
  "de",
  "la",
  "Mancha",
  "de",
  "cuyo",
  "nombre",
  "no",
  "quiero",
  "acordarme"
)
res69_2: List[String] = List(
  "En",
  "un",
  "lugar",
  "de",
  "la",
  "Mancha",
  "de",
  "cuyo",
  "nombre",
  "no",
  "quiero",
  "acordarme"
)

In [60]:
def flatten[A](l: List[List[A]]): List[A] = 
    ???

defined function flatten

In [61]:
def flatten[A](l: List[List[A]]): List[A] = 
    l.foldRight[List[A]](??? : List[A])
        ((head: List[A], tailSol: List[A]) => 
             ???)

defined function flatten

In [71]:
def flatten[A](l: List[List[A]]): List[A] = 
    l.foldRight[List[A]](Nil : List[A])
        ((head: List[A], tailSol: List[A]) => 
             head ++ tailSol)

defined function flatten

In [74]:
def flatten[A](l: List[List[A]]): List[A] = 
    l.foldLeft[List[A]](Nil : List[A])
        ((out: List[A], e: List[A]) => 
             out ++ e)

defined function flatten

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [53]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")

???

paragraph: List[String] = List("en un lugar", "de la Mancha")
res53_1: List[List[String]] = List(
  List("en", "un", "lugar"),
  List("de", "la", "Mancha")
)

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [54]:
// flatten


defined function flatten

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [76]:
def words(paragraph: List[String]): List[String] = 
    paragraph
        .map((sentence: String) => sentence.split(" ").toList)
        .flatten

defined function words

In [78]:
def words(paragraph: List[String]): List[String] = 
    paragraph
        .map(_.split(" "))
        .flatten

defined function words

In [80]:
def words(paragraph: List[String]): List[String] = 
    paragraph
        .flatMap(_.split(" "))

defined function words

In [82]:
def map[A, B](l: List[A])(f: A => B): List[B] = ??? 

defined function map

In [83]:
def flatMap[A, B](l: List[A])(f: A => List[B]): List[B] =
    l.map(f).flatten

defined function flatMap

In [81]:
run(TestWords(words))

cmd55$Helper$TestWords:
words
- should work


This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [58]:
// flatMap 


defined function flatMap

![filter](../../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [59]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

### FlatMapping optional values

In [60]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

defined function flatMap

In [61]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in:
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe ???
        flatMap(Some(-5))(isPositiveEven) shouldBe ???
        flatMap(Some(0))(isPositiveEven) shouldBe ???
        flatMap(Some(4))(isPositiveEven) shouldBe ???

defined class TestFlatMapOption

In [62]:
run(TestFlatMapOption(flatMap))

cell61$Helper$TestFlatMapOption:
flatMap
- should work


## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [88]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in:
        lengths(paragraph1) shouldBe 
            List(2, 2, 5, 2, 2, 6, 
                 2, 4, 6, 2, 6, 9)

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [86]:
List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    .flatMap(sent => sent.split(" ").toList)
    .filter(word => word != "")
    .map(word => word.length)

res86: List[Int] = List(2, 2, 5, 2, 2, 6, 2, 4, 6, 2, 6, 9)

In [89]:
def lengthsM(paragraph: List[String]): List[Int] =
    paragraph
        .flatMap(_.split(" ").toList)
        .filter(_ != "")
        .map(_.length)

defined function lengthsM

In [91]:
def lengthsM(paragraph: List[String]): List[Int] =
    paragraph
        .flatMap(_.split(" ").toList)
        .map(_.length)
        .filter(_ != 0)

defined function lengthsM

In [93]:
def lengthsM(paragraph: List[String]): List[Int] =
    paragraph
        .flatMap(_.split(" ").toList : List[String])
        .map(_.length)
        .filter(_ != 0)

defined function lengthsM

In [94]:
def lengthsM(paragraph: List[String]): List[Int] =
    paragraph
        .flatMap(_.split(" ").toList.map(_.length) : List[Int])
        .filter(_ != 0)

defined function lengthsM

and this works, of course: 

In [95]:
run(TestLengths(lengthsM))

cmd88$Helper$TestLengths:
lengths
- should work


but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [71]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsHOF

In [73]:
run(TestLengths(lengthsHOF))

cell64$Helper$TestLengths:
lengths
- should work


This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [3]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsFC

In [77]:
run(TestLengths(lengthsFC))

cell64$Helper$TestLengths:
lengths
- should work
